In [1]:
pip install stockfish

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
import chess
from stockfish import Stockfish
import tkinter as tk
from tkinter import messagebox
import time

class ChessGUI:
    def __init__(self, root, elo=650):
        self.root = root
        self.elo = elo
        self.selected_piece = None
        self.board = chess.Board()
        self.white_perspective = True
        self.move_history = []  # Hamle geçmişi
        self.dragging_piece = None  # Sürüklenen taş bilgisi
        self.drag_data = {"x": 0, "y": 0, "item": None}  # Sürükleme verileri

        # Stockfish ayarları
        self.stockfish = Stockfish(path=r"Kodun bulunduğu klasörü buraya girin. Klasörün içinde https://stockfishchess.org/download sitesinden indirdiğiniz alt klasör olmalı. \\stockfish\\stockfish-windows-x86-64.exe")
        
        # Güncel Stockfish metodları
        self.stockfish.set_depth(10)
        self.stockfish.set_elo_rating(elo)
        
        # GUI Ayarları
        self.root.title(f"Satranç Botu - ELO {elo}")
        self.canvas = tk.Canvas(root, width=480, height=480)
        self.canvas.pack()
        
        # Kontrol paneli
        self.control_frame = tk.Frame(root)
        self.control_frame.pack(pady=10)
        
        self.suggest_btn = tk.Button(self.control_frame, text="Hamle Öner ve Oyna", command=self.play_recommendation)
        self.suggest_btn.pack(side="left", padx=5)
        
        self.reset_btn = tk.Button(self.control_frame, text="Oyunu Sıfırla", command=self.reset_game)
        self.reset_btn.pack(side="left", padx=5)
        
        self.flip_btn = tk.Button(self.control_frame, text="Tahtayı Çevir", command=self.flip_board)
        self.flip_btn.pack(side="left", padx=5)
        
        self.undo_btn = tk.Button(self.control_frame, text="Geri Al", command=self.undo_move)  # Yeni buton
        self.undo_btn.pack(side="left", padx=5)
        
        self.status_label = tk.Label(root, text="", font=('Arial', 12))
        self.status_label.pack()
        
        # Sürükleme olayları
        self.canvas.bind("<ButtonPress-1>", self.on_drag_start)
        self.canvas.bind("<B1-Motion>", self.on_drag_move)
        self.canvas.bind("<ButtonRelease-1>", self.on_drag_stop)
        
        self.draw_board()
        self.update_display()
        self.add_board_labels()
        parameters = self.stockfish.get_parameters()
        print(parameters)
    
    def draw_board(self):
        self.canvas.delete("all")
        colors = ["#FFFFFF", "#90EE90"]  # Yeni renk teması
        
        offset = 35
        for row in range(8):
            for col in range(8):
                if self.white_perspective:
                    display_row = row
                    display_col = col
                else:
                    display_row = 7 - row
                    display_col = 7 - col
                
                x1 = display_col * 50 + offset
                y1 = display_row * 50 + offset
                x2 = x1 + 50
                y2 = y1 + 50
                self.canvas.create_rectangle(
                    x1, y1, x2, y2,
                    fill=colors[(row+col) % 2],
                    outline=""
                )
        
        self.add_board_labels()
        self.update_display()

    def add_board_labels(self):
        offset = 35
        letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
        
        # Yatay etiketler (a-h)
        for col in range(8):
            if self.white_perspective:
                x = col * 50 + offset + 25
                letter = letters[col]
            else:
                x = (7 - col) * 50 + offset + 25  # Sütunları ters çevir
                letter = letters[col]  # letters[7 - col] olmamalı
            
            y = 8 * 50 + offset + 15
            self.canvas.create_text(x, y, text=letter, font=('Arial', 12))
        
        # Dikey etiketler (1-8)
        for row in range(8):
            physical_row = 7 - row if self.white_perspective else row
            number = str(physical_row + 1)
            x = offset - 15
            y = row * 50 + offset + 25
            self.canvas.create_text(x, y, text=number, font=('Arial', 12))

    def update_display(self):
        self.canvas.delete("pieces")
        piece_symbols = {
            'r': '♜', 'n': '♞', 'b': '♝', 'q': '♛', 'k': '♚', 'p': '♟',
            'R': '♖', 'N': '♘', 'B': '♗', 'Q': '♕', 'K': '♔', 'P': '♙'
        }
        offset = 35
        
        for square in chess.SQUARES:
            piece = self.board.piece_at(square)
            if piece:
                # Gerçek koordinatları hesapla
                file = chess.square_file(square)
                rank = chess.square_rank(square)
                
                # Perspektife göre görüntüleme koordinatları
                if self.white_perspective:
                    display_file = file
                    display_rank = 7 - rank
                else:
                    display_file = 7 - file
                    display_rank = rank
                
                x = display_file * 50 + offset + 25
                y = display_rank * 50 + offset + 25
                self.canvas.create_text(
                    x, y,
                    text=piece_symbols[piece.symbol()],
                    font=('Arial', 24),
                    tags="pieces"
                )
        
        turn = "Beyaz" if self.board.turn else "Siyah"
        self.status_label.config(text=f"Sıra: {turn} | Perspektif: {'Beyaz' if self.white_perspective else 'Siyah'}")

    def on_square_click(self, event):
        offset = 35
        raw_col = (event.x - offset) // 50
        raw_row = (event.y - offset) // 50
        
        # Perspektife göre gerçek koordinatları hesapla
        if self.white_perspective:
            col = raw_col
            row = 7 - raw_row
        else:
            col = 7 - raw_col
            row = raw_row
        
        if col < 0 or col > 7 or row < 0 or row > 7:
            return
        
        square = chess.square(col, row)
        
        if self.selected_piece is None:
            piece = self.board.piece_at(square)
            if piece:
                self.selected_piece = square
                self.highlight_move(square)
        else:
            move = chess.Move(self.selected_piece, square)
            if move in self.board.legal_moves:
                self.board.push(move)
            self.selected_piece = None
            self.draw_board()

    def highlight_move(self, from_sq, to_sq):
        """Hamle animasyonu için vurgulama"""
        offset = 35
        duration = 0.5
        
        # Başlangıç pozisyonu
        from_col = chess.square_file(from_sq)
        from_row = 7 - chess.square_rank(from_sq)
        if not self.white_perspective:
            from_col = 7 - from_col
            from_row = 7 - from_row
        
        # Bitiş pozisyonu
        to_col = chess.square_file(to_sq)
        to_row = 7 - chess.square_rank(to_sq)
        if not self.white_perspective:
            to_col = 7 - to_col
            to_row = 7 - to_row
        
        # Vurgulama renkleri
        self.canvas.create_rectangle(
            from_col*50 + offset, from_row*50 + offset,
            (from_col+1)*50 + offset, (from_row+1)*50 + offset,
            fill="#FFD700",  # Açık sarı
            outline="",
            stipple="gray50",  # Noktalı efekt
            tags="highlight"
        )
        self.canvas.create_rectangle(
            to_col*50 + offset, to_row*50 + offset,
            (to_col+1)*50 + offset, (to_row+1)*50 + offset,
            fill="#FFA500",  # Açık turuncu
            outline="",
            stipple="gray50",
            tags="highlight"
        )


        
        self.canvas.update()
        self.root.after(300, lambda: self.canvas.delete("highlight"))
        time.sleep(duration)
        self.draw_board()


    def on_drag_start(self, event):
        """Sürükleme başlangıcı"""
        offset = 35
        col = (event.x - offset) // 50
        row = (event.y - offset) // 50
        
        if self.white_perspective:
            square = chess.square(col, 7 - row)
        else:
            square = chess.square(7 - col, row)
        
        piece = self.board.piece_at(square)
        if piece and piece.color == self.board.turn:
            self.dragging_piece = {
                "square": square,
                "symbol": piece.symbol(),
                "x": event.x,
                "y": event.y
            }
            self.drag_data["item"] = self.canvas.create_text(
                event.x, event.y,
                text=piece.unicode_symbol(),
                font=('Arial', 24),
                tags="drag_piece"
            )

    def on_drag_move(self, event):
        """Sürükleme devam ediyor"""
        if self.drag_data["item"]:
            self.canvas.move(self.drag_data["item"], 
                            event.x - self.drag_data["x"], 
                            event.y - self.drag_data["y"])
            self.drag_data["x"] = event.x
            self.drag_data["y"] = event.y

    def on_drag_stop(self, event):
        """Sürükleme bitişi"""
        if not self.dragging_piece:
            return
        
        offset = 35
        from_sq = self.dragging_piece["square"]
        to_col = (event.x - offset) // 50
        to_row = (event.y - offset) // 50
        
        if self.white_perspective:
            to_sq = chess.square(to_col, 7 - to_row)
        else:
            to_sq = chess.square(7 - to_col, to_row)
        
        move = chess.Move(from_sq, to_sq)
        if move in self.board.legal_moves:
            self.board.push(move)
            self.move_history.append(self.board.fen())
            self.highlight_move(from_sq, to_sq)
        
        self.canvas.delete("drag_piece")
        self.dragging_piece = None
        self.drag_data = {"x": 0, "y": 0, "item": None}
        self.update_display()

    
    def play_recommendation(self):
        if self.board.is_game_over():
            messagebox.showinfo("Oyun Bitti", self.board.result())
            return
            
        self.stockfish.set_fen_position(self.board.fen())
        best_move = self.stockfish.get_best_move()
        
        try:
            from_sq = chess.parse_square(best_move[:2])
            to_sq = chess.parse_square(best_move[2:])
            self.board.push(chess.Move.from_uci(best_move))
            self.move_history.append(self.board.fen())
            self.status_label.config(text=f"Oynanan hamle: {best_move.upper()}", fg="blue")
            self.highlight_move(from_sq, to_sq)
        except:
            messagebox.showerror("Hata", "Geçersiz hamle önerisi!")

    def reset_game(self):
        self.board.reset()
        self.status_label.config(text="Oyun sıfırlandı!", fg="red")
        self.draw_board()

    def flip_board(self):  # Yeni eklenen fonksiyon
        self.white_perspective = not self.white_perspective
        self.draw_board()
        


    def undo_move(self):
        """Son hamleyi geri al"""
        if len(self.move_history) > 1:
            self.move_history.pop()
            self.board.set_fen(self.move_history[-1])
            self.draw_board()
            self.status_label.config(text="Son hamle geri alındı!", fg="orange")
        else:
            self.status_label.config(text="Geri alınacak hamle yok!", fg="red")

if __name__ == "__main__":
    root = tk.Tk()
    gui = ChessGUI(root, elo=1218)
    root.mainloop()

Stockfish motoru aranıyor veya indiriliyor...


Exception ignored in: <function Stockfish.__del__ at 0x000002871471A160>
Traceback (most recent call last):
  File "C:\Users\velii\AppData\Roaming\Python\Python313\site-packages\stockfish\models.py", line 757, in __del__
    if self._stockfish.poll() is None:
AttributeError: 'Stockfish' object has no attribute '_stockfish'
